# Austrian energy system Tutorial Part 1: Building an Energy Model (in R)

For information on how to install *MESSAGEix*, please refer to the [Installation page](https://docs.messageix.org/en/stable/#getting-started), and for getting the *MESSAGEix* tutorials, please follow the steps mentioned in [Tutorials](https://docs.messageix.org/en/stable/tutorials.html).

Please refer to the [user guidelines](https://docs.messageix.org/en/stable/notice.html) for additional information on using *MESSAGEix*, including the recommended citation and how to name new models.

**Pre-requisites**
- You have the *MESSAGEix* framework installed and working

**Structure of these tutorials.** After having run this baseline tutorial, you are able to start with any of the other tutorials, but we recommend to follow the order below for going through the information step-wise:

1. Prepare the base model version (Python: ``austria.ipynb``, R: ``R_austria.ipynb``)
2. Plot the results of the baseline runs (Python: ``austria_load_scenario.ipynb``, R: ``R_austria_load_scenario.ipynb``)

Currently only available in Python:
3. Run a single policy scenario (``austria_single_policy.ipynb``)
4. Run multiple policy scenarios. This tutorial has two notebooks: an introduction with some exercises and completed code for the exercises (exercises: ``austria_multiple_policies.ipynb``, answers: ``austria_multiple_policies-answers.ipynb``)

**Introduction**

In this notebook, we will build a model of the Austrian energy system from scratch. The process will involve defining our model's time horizon and spatial extent, and then populating the model with data associated with model parameters. Once we have a baseline model, we will then move on to investigating policy scenarios.

We will be populating different kinds of parameters including:

### Economic Parameters

- `interestrate`
- `demand`

### Technology Parameters

#### Engineering Parameters

- `input`
- `output`
- `technical_lifetime`
- `capacity_factor`


#### Technoeconomic Parameters

- `inv_cost`
- `fix_cost`
- `var_cost`

### Dynamic Behavior Parameters

- `bound_activity_up`
- `bound_activity_lo`
- `bound_new_capacity_up`
- `initial_activity_up`
- `growth_activity_up`

### Emissions

- `emission_factor`

A full list of parameters can be found on the (internal) [MESSAGEix documentation website](http://ienecat.iiasa.ac.at:8787/message_ix_doc). This website can be built in the future from the repository that you are using. 

## The Final Product

At the completion of this exercise, we will have developed an energy model that is comprised of the below Reference Energy System (RES):

![title](austria.png)



## Setup

We begin by loading the ``rmessageix`` package.
We also load the common R package ``dplyr``, which will be used for some data manipulation.

In [ ]:
# Load dplyr, used for data manipulation in R
library(dplyr)

# Load reticulate, used to access the Python API from R
library(reticulate)

# Import ixmp and message_ix, just as in Python
ixmp <- import("ixmp")
message_ix <- import("message_ix")

These two objects named ``ixmp`` and ``message_ix`` can now be used to access all the features of the API (application programming interface) provided by the Python packages.

In [ ]:
# launch the IX modeling platform using the local default database
mp <- ixmp$Platform()

In [ ]:
model <- "Austrian energy model"
scen <- "baseline"
annot <- "developing a stylized energy system model for illustration and testing" 

scenario <- message_ix$Scenario(mp, model, scen, version='new', annotation=annot)

## Time and Spatial Detail

The model includes the time periods 2010, 2020, 2030 and 2040.

In [ ]:
horizon = as.integer(seq(2010, 2040, 10))
firstyear = as.integer(horizon[1])

In [ ]:
scenario$add_set("year", horizon)
scenario$add_set("cat_year", c("firstmodelyear", firstyear) )

In [ ]:
country = "Austria"
scenario$add_set("node", country)
scenario$add_set("lvl_spatial", "country")
scenario$add_set("map_spatial_hierarchy", c("country", country, "World"))
scenario$add_set("mode", "standard")

In [ ]:
scenario$add_set("commodity", c("electricity", "light", "other_electricity") )
scenario$add_set("level", c("secondary", "final", "useful") )

## Economic Parameters

In [ ]:
scenario$add_par("interestrate", horizon, value=0.05, unit='-')

In [ ]:
beta = 0.7 
gdp = c(1., 1.21631, 1.4108, 1.63746)
demand = gdp ** beta

## Technologies

In [ ]:
plants = c(    "coal_ppl", 
    "gas_ppl", 
    "oil_ppl", 
    "bio_ppl", 
    "hydro_ppl",
    "wind_ppl", 
    "solar_pv_ppl" # actually primary -> final
)
secondary_energy_techs = c(plants,'import')

final_energy_techs = c('electricity_grid')

lights = c(
    "bulb", 
    "cfl" 
)
useful_energy_techs = c(lights ,'appliances')

In [ ]:
technologies = c(secondary_energy_techs, final_energy_techs,  useful_energy_techs)
scenario$add_set("technology", technologies)

In [ ]:
demand_per_year = 55209. / 8760 # from IEA statistics
elec_demand = data.frame(
    node = rep(country,4),
    commodity = rep('other_electricity',4),
    level = rep('useful',4),
    year = horizon,
    time = rep('year',4),
    value = demand_per_year * demand,
    unit = rep('GWa',4),
    row.names = horizon
)
scenario$add_par("demand", elec_demand)

demand_per_year = 6134. / 8760 # from IEA statistics
light_demand = data.frame(
    node = country,
    commodity = 'light',
    level = 'useful',
    year = horizon,
    time = 'year',
    value = demand_per_year * demand,
    unit = 'GWa'
)
scenario$add_par("demand", light_demand)

### Engineering Parameters

In [ ]:
year_pairs = as.matrix(expand.grid(horizon,horizon) %>%
                         rowwise() %>%
                         filter(Var2 >= Var1) %>%
                         arrange(Var1))
vintage_years <- year_pairs[,1]
act_years <- year_pairs[,1]

In [ ]:
base_input = data.frame(
    node_loc = country,
    year_vtg = vintage_years,
    year_act = act_years,
    mode = 'standard',
    node_origin = country,
    commodity = 'electricity',
    time = 'year',
    time_origin = 'year'
)

grid = data.frame(base_input,data.frame(
        technology = 'electricity_grid',
        level = 'secondary',
        value = 1.0,
        unit = '%'
        ))
scenario$add_par("input", grid)

bulb = data.frame(base_input,data.frame(
        technology = 'bulb',
        level = 'final',
        value = 1.0,
        unit = '%'
        ))
scenario$add_par("input", bulb)

cfl = data.frame(base_input,data.frame(
        technology = 'cfl',
        level = 'final',
        value = 0.3, 
        unit = '%'
        ))
scenario$add_par("input", cfl)

app = data.frame(base_input,data.frame(
        technology = 'appliances',
        level = 'final',
        value = 1.0,
        unit = '%'
        ))
scenario$add_par("input", app)

In [ ]:
base_output = data.frame(
    node_loc = country,
    year_vtg = vintage_years,
    year_act = act_years,
    mode = 'standard',
    node_dest = country,
    time = 'year',
    time_dest = 'year', 
    unit = '%'
)

imports = data.frame(base_output,data.frame( technology='import', commodity='electricity', 
                  level='secondary', value=1.))
scenario$add_par('output', imports)

grid = data.frame(base_output,data.frame( technology='electricity_grid', commodity='electricity', 
               level='final', value=0.873))
scenario$add_par('output', grid)

bulb = data.frame(base_output,data.frame( technology='bulb', commodity='light', 
               level='useful', value=1.))
scenario$add_par('output', bulb)

cfl = data.frame(base_output,data.frame( technology='cfl', commodity='light', 
              level='useful', value=1.))
scenario$add_par('output', cfl)

app = data.frame(base_output,data.frame( technology='appliances', commodity='other_electricity', 
              level='useful', value=1.))
scenario$add_par('output', app)

coal = data.frame(base_output,data.frame( technology='coal_ppl', commodity='electricity', 
               level='secondary', value=1.))
scenario$add_par('output', coal)

gas = data.frame(base_output,data.frame( technology='gas_ppl', commodity='electricity', 
              level='secondary', value=1.))
scenario$add_par('output', gas)

oil = data.frame(base_output,data.frame( technology='oil_ppl', commodity='electricity', 
              level='secondary', value=1.))
scenario$add_par('output', oil)

bio = data.frame(base_output,data.frame( technology='bio_ppl', commodity='electricity', 
              level='secondary', value=1.))
scenario$add_par('output', bio)

hydro = data.frame(base_output,data.frame( technology='hydro_ppl', commodity='electricity', 
                level='secondary', value=1.))
scenario$add_par('output', hydro)

wind = data.frame(base_output,data.frame( technology='wind_ppl', commodity='electricity', 
               level='secondary', value=1.))
scenario$add_par('output', wind)

solar_pv = data.frame(base_output,data.frame( technology='solar_pv_ppl', commodity='electricity', 
                   level='final', value=1.))
scenario$add_par('output', solar_pv)

In [ ]:
# NB this and the following cell can be modified to a syntax like the above

base_technical_lifetime = data.frame(
    node_loc = country,
    year_vtg = horizon,
    unit = 'y'
)

lifetimes = list(
    coal_ppl = 40,
    gas_ppl =  30,
    oil_ppl = 30,
    bio_ppl =  30,
    hydro_ppl = 60,
    wind_ppl = 20,
    solar_pv_ppl = 20,
    bulb = 1,
    cfl = 10
)
base_technical_lifetime

for (i in seq_along(lifetimes)){
    df = data.frame(base_technical_lifetime, technology=names(lifetimes[i]), value=lifetimes[[i]])
    scenario$add_par('technical_lifetime', df)
}

In [ ]:
base_capacity_factor = data.frame(
    node_loc = country,
    year_vtg = vintage_years,
    year_act = act_years,
    time = 'year',
    unit = '%'
)

capacity_factor = list(
    coal_ppl = 0.85,
    gas_ppl =  0.75,
    oil_ppl = 0.75,
    bio_ppl = 0.75,
    hydro_ppl = 0.5,
    wind_ppl = 0.2,
    solar_pv_ppl = 0.15,
    bulb = 0.1, 
    cfl =  0.1
)
    
for (i in seq_along(capacity_factor)){
    df = data.frame(base_capacity_factor, technology=names(capacity_factor[i]), value=capacity_factor[[i]])
    scenario$add_par('capacity_factor', df)
}

### Technoeconomic Parameters

In [ ]:
base_inv_cost = data.frame(
    node_loc = country,
    year_vtg = horizon,
    unit = 'USD/GWa'
)

# in $ / kW
costs = list(
    coal_ppl = 1500,
    gas_ppl =   870,
    oil_ppl =  950,
    hydro_ppl = 3000,
    bio_ppl =  1600,
    wind_ppl = 1100,
    solar_pv_ppl = 4000,
    bulb = 5,
    cfl =  900
)

for (i in seq_along(costs)){
    df = data.frame(base_inv_cost, technology=names(costs[i]), value=costs[[i]] * 1e6)
    scenario$add_par('inv_cost', df)
}

In [ ]:
base_fix_cost = data.frame(
    node_loc = country,
    year_vtg = vintage_years,
    year_act = act_years,
    unit = 'USD/GWa'
)

# in $ / kW
costs = list(
    coal_ppl = 40,
    gas_ppl =   25,
    oil_ppl =  25,
    hydro_ppl = 60,
    bio_ppl =  30,
    wind_ppl = 40,
    solar_pv_ppl = 25
)

for (i in seq_along(costs)){
    df = data.frame(base_fix_cost, technology=names(costs[i]), value=costs[[i]] * 1e6)
    scenario$add_par('fix_cost', df)
}

In [ ]:
base_var_cost = data.frame(
    node_loc = country,
    year_vtg = vintage_years,
    year_act = act_years,
    mode = 'standard',
    time = 'year',
    unit = 'USD/GWa'
)

# in $ / MWh
costs = list(
    coal_ppl = 24.4,
    gas_ppl =   42.4,
    oil_ppl =   77.8,
    bio_ppl =  48.2,
    electricity_grid = 47.8
)
    
for (i in seq_along(costs)){
    df = data.frame(base_var_cost, technology=names(costs[i]), value=costs[[i]] * 8760 * 1e3)
    scenario$add_par('var_cost', df)
}

## Dynamic Behavior Parameters

In [ ]:
base_growth = data.frame(
    node_loc = country,
    year_act = horizon[-1],
    value = 0.05,
    time = 'year',
    unit = '%'
)

growth_technologies = c(
    "coal_ppl", 
    "gas_ppl", 
    "oil_ppl", 
    "bio_ppl", 
    "hydro_ppl",
    "wind_ppl", 
    "solar_pv_ppl", 
    "cfl",
    "bulb"
)

for (tec in (growth_technologies)){
    df = data.frame(base_growth, technology= tec)
    scenario$add_par('growth_activity_up', df)
}

In [ ]:
base_initial = data.frame(
    node_loc = country,
    year_act = horizon[-1],
    time = 'year',
    unit = '%'
)

for (tec in (lights)){
    df = data.frame(base_initial, technology= tec, value = 0.01 *light_demand$value[light_demand$year %in% c(2020,2030,2040)])
    scenario$add_par('initial_activity_up', df)
}

In [ ]:
base_activity = data.frame(
    node_loc = country,
    year_act = as.integer(2010),
    mode = 'standard',
    time = 'year',
    unit = 'GWa'
)

# in GWh - from IEA Electricity Output
activity = list(
    coal_ppl = 7184,
    gas_ppl =  14346,
    oil_ppl =  1275,
    hydro_ppl = 38406,
    bio_ppl =  4554,
    wind_ppl = 2064,
    solar_pv_ppl = 89,
    import = 2340,
    cfl = 0
)

for (i in seq_along(activity)){
    df = data.frame(base_activity, technology=names(activity[i]), value=activity[[i]]/8760)
    scenario$add_par('bound_activity_up', df)
    scenario$add_par('bound_activity_lo', df)
}

In [ ]:
base_capacity = data.frame(
    node_loc = country,
    year_vtg = as.integer(2010),
    unit = 'GWa'
)

a = as.data.frame(t(as.data.frame(activity))) %>% rename(act = V1)
b = as.data.frame(t(as.data.frame(capacity_factor)))%>% rename(cf = V1) 
capacity = left_join(
    a%>% mutate(technology = row.names(a)),
    b%>% mutate(technology = row.names(b))) %>% 
  mutate(value = (act / 8760 / cf)) %>% 
  filter(!is.na(value))

df = data.frame(base_capacity,capacity)
scenario$add_par('bound_new_capacity_up', df)

In [ ]:
base_activity = data.frame(
    node_loc = country,
    year_act = horizon[-1],
    mode = 'standard',
    time = 'year',
    unit = 'GWa'
)

# in GWh - base value from IEA Electricity Output
keep_activity = list(
    hydro_ppl = 38406,
    bio_ppl =  4554,
    import = 2340
)
    
for (i in seq_along(keep_activity)){
    df = data.frame(base_activity, technology=names(keep_activity[i]), value=keep_activity[[i]])
    scenario$add_par('bound_activity_up', df)
}

## Emissions

In [ ]:
scenario$add_set("emission", "CO2")
scenario$add_cat('emission', 'GHGs', 'CO2')

In [ ]:
base_emissions = data.frame(
    node_loc = country,
    year_vtg = vintage_years,
    year_act = act_years,
    mode = 'standard',
    unit = 'kg/kWa' # actually is tCO2/GWa
)

# units: tCO2/MWh
emissions = list(
    coal_ppl =  c('CO2', 0.854),
    gas_ppl =   c('CO2', 0.339),
    oil_ppl =   c('CO2', 0.57)
)
    
for (i in seq_along(emissions)){
    df = data.frame(base_emissions, technology=names(emissions[i]),emission = emissions[[i]][1], value = (as.numeric(emissions[[i]][2])* 8760. * 1000) )
    scenario$add_par('emission_factor', df)
}

## Commit the datastructure and solve the model

In [ ]:
comment = 'initial commit for Austria model'
scenario$commit(comment)
scenario$set_as_default()

In [ ]:
scenario$solve('MESSAGE')

In [ ]:
scenario$var('OBJ')['lvl']

In [ ]:
mp$close_db()